## NMEA Parsing, in Python

In [1]:
import json
import nmea_parser as NMEAParser

---------------------
nmea_parser NOT running as main, probably imported.
---------------------


In [4]:
samples = [
    "$IIRMC,092551,A,1036.145,S,15621.845,W,04.8,317,,10,E,A*0D\r\n",
    "$IIMWV,088,T,14.34,N,A*27\r\n",
    "$IIVWR,148.,L,02.4,N,01.2,M,04.4,K*XX\r\n",
    "$IIVTG,054.7,T,034.4,M,005.5,N,010.2,K,A*XX\r\n",
    "$GPTXT,01,01,02,u-blox ag - www.u-blox.com*50\r\n",
    "$GPRMC,183333.000,A,4047.7034,N,07247.9938,W,0.66,196.21,150912,,,A*7C\r\n",
    "$IIGLL,3739.854,N,12222.812,W,014003,A,A*49\r\n"
]

In [5]:

# akeu = sex_to_dec("12", "34.XX")

for sentence in samples:
    print("Parsing {}".format(sentence))
    try:
        nmea_obj = NMEAParser.parse_nmea_sentence(sentence)
        try:
            print('Parsed Object: {}'.format(json.dumps(nmea_obj, indent=2)))
        except TypeError as type_error:
            print('TypeError: {}'.format(type_error))
            print('Parsed Object (raw): {}'.format(nmea_obj))
    except Exception as ex:
        print("Ooops! {}, {}".format(type(ex), ex))
    print("----------------------------------")


Parsing $IIRMC,092551,A,1036.145,S,15621.845,W,04.8,317,,10,E,A*0D

Parsed Object: {
  "type": "rmc",
  "parsed": {
    "valid": "true",
    "position": {
      "latitude": -10.602416666666667,
      "longitude": -156.36408333333333
    },
    "sog": 4.8,
    "cog": 317.0,
    "declination": 10.0,
    "type": "autonomous"
  }
}
----------------------------------
Parsing $IIMWV,088,T,14.34,N,A*27

Ooops! <class 'nmea_parser.NoParserException'>, No parser exists (yet) for MWV
----------------------------------
Parsing $IIVWR,148.,L,02.4,N,01.2,M,04.4,K*XX

Ooops! <class 'nmea_parser.InvalidChecksumException'>, Invalid checksum for $IIVWR,148.,L,02.4,N,01.2,M,04.4,K*XX
----------------------------------
Parsing $IIVTG,054.7,T,034.4,M,005.5,N,010.2,K,A*XX

Ooops! <class 'nmea_parser.InvalidChecksumException'>, Invalid checksum for $IIVTG,054.7,T,034.4,M,005.5,N,010.2,K,A*XX
----------------------------------
Parsing $GPTXT,01,01,02,u-blox ag - www.u-blox.com*50

Parsed Object: {
  "type": 

## Full Sample, RMC Parsing

In [6]:
rmc_data = "$GPRMC,183333.000,A,4047.7034,N,07247.9938,W,0.66,196.21,150912,,,A*7C\r\n"

### Validation

In [7]:
valid = True

In [12]:
if rmc_data[0] != '$':
    valid = False
    print("String does not begin with '$', not valid")
else:
    print("Start of String OK, moving on.")

Start of String OK, moving on.


In [16]:
if valid:
    if rmc_data[-2:] != "\r\n":
        valid = False
        print("Bad string termination")
    else:
        print("String termination OK, moving on.")

String termination OK, moving on.


In [26]:
if valid:
    string_to_validate = rmc_data[1:-5]
    checksum = rmc_data[-4:-2]
    print("Data to validate: {} against {}".format(string_to_validate, checksum))

Data to validate: GPRMC,183333.000,A,4047.7034,N,07247.9938,W,0.66,196.21,150912,,,A against 7C


In [25]:
if valid:
    cs = 0
    char_array = list(string_to_validate)
    for c in range(len(string_to_validate)):
        cs = cs ^ ord(char_array[c])
        print ("Char {} (0x{:02x}) -> CheckSum now 0x{:02x}".format(char_array[c], ord(char_array[c]), cs))
    original_cs = int(checksum, 16)
    if original_cs != cs:
        valid = False
        print("Invalid Checksum: Found {:02x}, expected {:02x}".format(cs, original_cs))
    else:
        print("Checksum OK, moving on")
    

Char G (0x47) -> CheckSum now 0x47
Char P (0x50) -> CheckSum now 0x17
Char R (0x52) -> CheckSum now 0x45
Char M (0x4d) -> CheckSum now 0x08
Char C (0x43) -> CheckSum now 0x4b
Char , (0x2c) -> CheckSum now 0x67
Char 1 (0x31) -> CheckSum now 0x56
Char 8 (0x38) -> CheckSum now 0x6e
Char 3 (0x33) -> CheckSum now 0x5d
Char 3 (0x33) -> CheckSum now 0x6e
Char 3 (0x33) -> CheckSum now 0x5d
Char 3 (0x33) -> CheckSum now 0x6e
Char . (0x2e) -> CheckSum now 0x40
Char 0 (0x30) -> CheckSum now 0x70
Char 0 (0x30) -> CheckSum now 0x40
Char 0 (0x30) -> CheckSum now 0x70
Char , (0x2c) -> CheckSum now 0x5c
Char A (0x41) -> CheckSum now 0x1d
Char , (0x2c) -> CheckSum now 0x31
Char 4 (0x34) -> CheckSum now 0x05
Char 0 (0x30) -> CheckSum now 0x35
Char 4 (0x34) -> CheckSum now 0x01
Char 7 (0x37) -> CheckSum now 0x36
Char . (0x2e) -> CheckSum now 0x18
Char 7 (0x37) -> CheckSum now 0x2f
Char 0 (0x30) -> CheckSum now 0x1f
Char 3 (0x33) -> CheckSum now 0x2c
Char 4 (0x34) -> CheckSum now 0x18
Char , (0x2c) -> Che

### Validations OK
Now splitting the data

In [28]:
members = string_to_validate.split(',')
print("We have {} members:".format(len(members)))
for item in members:
    print(item)

We have 13 members
GPRMC
183333.000
A
4047.7034
N
07247.9938
W
0.66
196.21
150912


A


In [29]:
if len(members[0]) != 5:
    print("Bad length for sentence prefix and ID")
else:
    device_prefix = members[0][0:2]
    sentence_id = members[0][-3:]
    print("Device Prefix is {}, Sentence ID is {}".format(device_prefix, sentence_id))

Device Prefix is GP, Sentence ID is RMC
